# Preprocess the Herschel photometry
### Ramsey Karim, July 26, 2022

This code serves the same purpose as Tracy's IDL preprocessing routines, but is written specifically for the HGBS Lupus I moonlight corrected images since they do not come in the standard archive format and do not have error maps on the exact same grids.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os.path
import glob

img_stub = "-image.fits"
filename_list = glob.glob("*" + img_stub)
filename_dict = {f.replace(img_stub, ''): os.path.abspath(f) for f in filename_list}

del filename_list

for f in filename_dict:
    print(f, "->", filename_dict[f])

SPIRE250um -> /an/sgra~raid/filaments/LupusI-HGBcorrected/processed/SPIRE250um-image.fits
SPIRE500um -> /an/sgra~raid/filaments/LupusI-HGBcorrected/processed/SPIRE500um-image.fits
PACS160um -> /an/sgra~raid/filaments/LupusI-HGBcorrected/processed/PACS160um-image.fits
SPIRE350um -> /an/sgra~raid/filaments/LupusI-HGBcorrected/processed/SPIRE350um-image.fits


In [3]:
# Regrid the maps to the 500um grid
from astropy.io import fits
from reproject import reproject_interp

ref_band = 'SPIRE500um'
ref_header = fits.getheader(filename_dict[ref_band])

raw_band = 'SPIRE350um'
regridded_img = reproject_interp(filename_dict[raw_band], ref_header, return_footprint=False)

In [4]:
# Save the regridded image
from astropy.wcs import WCS
savename = filename_dict[raw_band].replace('.fits', '-remapped.fits')
header = fits.getheader(filename_dict[raw_band])
header.update(WCS(ref_header).to_header())
header['HISTORY'] = f"Regridded to {ref_band} grid by Ramsey Karim, 2022-07-26"
new_hdu = fits.PrimaryHDU(data=regridded_img, header=header)
new_hdu.writeto(savename)

For the convolution, I need the beam sizes of the maps. These are not listed in the FITS headers of these particular files, but we can get them from the [PACS Observer's Manual](https://www.cosmos.esa.int/documents/12133/996891/PACS+Observers'+Manual) (Section 3.1) and the [SPIRE Handbook](https://www.cosmos.esa.int/documents/12133/1035800/The+Herschel+Explanatory+Supplement%2C%20Volume+IV+-+THE+SPECTRAL+AND+PHOTOMETRIC+IMAGING+RECEIVER+%28SPIRE%29/c36d074d-32b4-48ec-b13f-4ca320788df3) (Section 5, Table 5.2 in version 3.2).

For PACS, I am assuming that all observations were taken at a scan speed of 60 arcsec/sec and in Parallel Mode, since this has been the case for almost all other observations we've processed. The FITS files given to us do not list the scan velocity or observing mode.
For SPIRE, I will just take the circular beam FWHMs that Tracy put in his ConvolveHerschelImages.pro code: 18.1, 24.9, and 36.4 arcsec.

In [5]:
from radio_beam import Beam
from astropy.convolution import convolve
from astropy import units as u

beam_params_dict = {
    # major (as), minor (as), PA (deg)
    'PACS70um': (12.16, 5.86, 63.0),
    'PACS160um': (15.65, 11.64, 53.4),
    
    'SPIRE250um': (18.1,),
    'SPIRE350um': (24.9,),
    'SPIRE500um': (36.4,),
}
def get_beam(band_stub):
    """
    :param band_stub: string like "PACS70um" which identifies the band
    :returns: Beam
    """
    beam_params = beam_params_dict[band_stub]
    if len(beam_params) > 1:
        # PACS, elliptical
        return Beam(major=beam_params[0]*u.arcsec, minor=beam_params[1]*u.arcsec, pa=beam_params[2]*u.deg)
    else:
        # SPIRE, circular
        return Beam(beam_params[0]*u.arcsec)

beam_raw = get_beam(raw_band)
beam_ref = get_beam(ref_band)

In [6]:
# Get the beam with which we should convolve the 250
conv_beam = beam_ref.deconvolve(beam_raw)

pixel_scale = (-1*ref_header['CDELT1']*u.deg).to(u.arcsec)
conv_kernel = conv_beam.as_kernel(pixel_scale)

conv_img = convolve(regridded_img, conv_kernel, preserve_nan=True)

In [7]:
# Save the convolved image
savename = filename_dict[raw_band].replace('.fits', '-remapped-conv.fits')
header = fits.getheader(filename_dict[raw_band])
header.update(WCS(ref_header).to_header())
header['HISTORY'] = f"Regridded to {ref_band} grid by Ramsey Karim, 2022-07-26"
header['HISTORY'] = f"Convolved to {ref_band} resolution by Ramsey Karim, 2022-07-26"
new_hdu = fits.PrimaryHDU(data=conv_img, header=header)
new_hdu.writeto(savename)

## I ran this code on July 26, 2022
It worked fine, it's a little messy but it does the job. I don't anticipate using this more than 2 more times (on the 160 and 350 micron maps when we get them).